In [1]:
 %pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pylab import *
from scipy import *
from scipy import stats, io
import numpy as np
import struct
import tables as tb
from attrdict import AttrDict
import matplotlib.pyplot as plt
import os as os
from phy.io import KwikModel
import codecs as codecs
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick

In [3]:
#----------------------------------------------------------------------------------------
#SAVING BINARY OBJECTS DATA
#need to automate data folder creation
#----------------------------------------------------------------------------------------
import pickle 

def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

# FUNCTIONS TO BUILD PSTHs

In [4]:
#----------------------------------------------------------------------------------------
# READ STIMULUS
#----------------------------------------------------------------------------------------
# Here we read the binary file with stimulus: 902 of 25 piezos x 1024 samples
# Text file has the type of stimulus: F sparse, C correlated, U uncorrelated
def read_stimulus(expe,meas):
    ## This function reads the stimulus binary file, reads the type of stimulus,
    ## and stores it in a matrix and a row vector
    
    bin_file = open(binname,'rb')
    read_data = np.fromfile(file=bin_file, dtype=np.float32)
    read_data = read_data.reshape((-1,25,10240)) # reshapes data assuming 25 whiskers, 10240 time bins
    txt_data = np.loadtxt(textname, dtype='S8') # Makes sure data type is text decoded
    txt_data = txt_data.view(np.chararray).decode('utf-8') # Makes sure data type is text decoded
    
    length2=0
    
    if expe==22 and meas[0:2] == 'm1':  #measurement 1 composed of two recorded
        length1=902
        length2 =93
        
    if expe == 20 and meas[0:2]=='m3': #there are 46 events in exp2 and 879 in exp3
        length1 = 46 
        length2 = 879
    
    if length2>0:
        txt_data = np.append(txt_data[0:length1], txt_data[0:length2])
        read_data = np.append(read_data[0:length1], read_data[0:length2],0)
        read_data = read_data.reshape((-1,25,10240)) # reshapes data assuming 25 whiskers, 10240 time bins
            
    close(binname)
    
    return read_data, txt_data

#----------------------------------------------------------------------------------------
# READKWIKINFO
#----------------------------------------------------------------------------------------
# We read the data of the output from klusterkwik: spike times and cluster-number of each
# cluster-number is in klustaviewa series (can be as high as 130 e.g.)
# Grupete stands for cluster groups! 2: good clusters, 1: multiunits, 0: unsorted, 3: noise
def readkwikinfo(kwik, grupete=3):
    model = KwikModel(kwik) # load kwik model from file
    spiketimes = model.spike_times # extract the absolute spike times
    clusters = model.cluster_groups # extract the cluster names
    sample_rate = model.sample_rate # extract sampling freq
    
    spikedata = {} # initialise dictionary
    for cluster in clusters.keys():
        clustergroup = clusters[cluster]
        if clustergroup==grupete: # only look at specified type of cluster, 0 = noise, 1 = MUA, 2 = GOOD, 3 = unsorted
            spiketimematrix = AttrDict({'spike_times': np.zeros(len(spiketimes[where(model.spike_clusters==cluster)]))})
            spiketimematrix.spike_times = spiketimes[where(model.spike_clusters==cluster)]
            spikedata[cluster] = spiketimematrix # data structure is a dictionary with attribute accessible spiketimes
            # attribute accessible means that spikedata.spike_times works, normal dictionaries would be spikedata[spike_times]
    
    model.close()
    
    return spikedata, sample_rate

#----------------------------------------------------------------------------------------
# READ VTAG
#----------------------------------------------------------------------------------------
def readVtag(Vtag1,stim,stimtype):
    ## The first task is to find the stimulus onset times for each whisker in each sweep in each direction
    start_and_stops = Vtag1[1:] - Vtag1[:-1]
    starts = (where(start_and_stops==1)[0]-2999)/float(sampling_freq) # time in seconds
    stops = (where(start_and_stops==-1)[0]+4110)/float(sampling_freq) # time in seconds
    
    stim_ret = stim[0:len(stops),:,:]
    stimtype_ret = stimtype[0:len(stops)]
    
    return stim_ret, stimtype_ret, starts, stops

#----------------------------------------------------------------------------------------
# BUILDS PSTH
#----------------------------------------------------------------------------------------
def BuildPSTH(stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops,exp,meas) :
## The first task is to find the stimulus onset times for each whisker in each sweep in each direction
    #stim, stimtype = read_stimulus()
    stim = stim[np.where(stimtype=='F')[0], :, :]
    starts = starts[np.where(stimtype=='F')[0]]
    stops = stops[np.where(stimtype=='F')[0]]
    
    stimtimes = {}
    for w in np.arange(25, dtype='int') :  
        timesUP = []
        timesDOWN = []
        for i in np.arange(len(stim), dtype='int') :
            indsUP = (np.where(stim[i, w, :]==1108.8889)[0]-1)[::2]
            # This finds all time points where the stim = 1108.8889, because each ramp has two 1108.8889 values
            # (on the way up and on the way down) we take every other index using [::2]
            timesUP.append(indsUP)
            indsDOWN = (np.where(stim[i, w, :]==-1108.8889)[0]-1)[::2]
            # This finds all time points where the stim = -1108.8889, because each ramp has two -1108.8889 values
            # (on the way up and on the way down) we take every other index using [::2]
            timesDOWN.append(indsDOWN)
        stimtimes[w] = timesUP, timesDOWN # stimtimes[whisker][0][:]=UP stimtimes[whisker][1][:]=DOWN
    
    # make an 'output dict'
    # the PSTH will be built on -tbefore:tafter
    hist_inds = {}
    PSTH = {}
    psth = dict()
    psth_times = dict()
    
    # Loop each neuron and get the spikes.
    for neuron in list(Spikes.keys()): 
        codename = 'exp'+ str(exp) + '_' + str(meas) + '_c' + str(neuron)
        
        psth = AttrDict({'clusnum': neuron,'exp' : int(exp) , 'meas': int(meas[1]) , 'shank': int(meas[3])})
        
        psth.update(AttrDict({'psth_counts': [] , 'psth_times': [] , 'psth_length': [t_before,t_after] }))
        
        psth['psth_counts'], psth['psth_times'] = PSTH_spikes(stim, stimtype, stimtimes, Spikes[neuron].spike_times, sampling_freq, t_before, t_after, starts, stops)
        
        PSTH[codename] = psth
       
    return PSTH

#----------------------------------------------------------------------------------------

def PSTH_spikes(stimulation, stimtype, stimtimes, spikes, samp, t_before, t_after, starts, stops):
    """
    stimulation   : a list of numpy arrays with a n*t stimulus inside
    stimtimes     : a list of the times the stimulus occurred for each whisker 
    spikes        : an array that contains the spike times (s)
    Vtag1         : synchronises stimulus with spike times
    samp          : sampling rate of the stimulation (Hz)
    t_before      : duration before the stim (positive, s)
    t_after       : duration after the stim (positive, s)
    starts        : the start of the F sweeps
    stops         : the stops of the F sweeps
    """
    
    stim_samp = 1/.0009997575757
    
    PSTH_spike_counts = {}
    for w in np.arange(25, dtype='int') :
        spikecountsup = 0
        spikecountsdown = 0
        for i in np.arange(len(stimulation), dtype='int') : 
            for x in np.arange(len(stimtimes[w][0][i]), dtype='int') :  # we must look at the number of stimulations per whisker per stimulation block and this is no longer 4        
                timesUP = starts[i] + stimtimes[w][0][i][x]/stim_samp # stimtimes is now stimtimes[whisker][0 for UP, 1 for DOWN][stimsweep][trial]
                spikecountsup += len(spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)]) # count spikes that are within PSTH window of stimtimes
            for y in np.arange(len(stimtimes[w][1][i]), dtype='int') : # for the DOWN stimuli we must have a separate loop because they also are now randomly distributed and not 4
                timesDOWN = starts[i] + stimtimes[w][1][i][y]/stim_samp                
                spikecountsdown += len(spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)])
        PSTH_spike_counts[w] = spikecountsup, spikecountsdown
    
    hist_inds = {} #same changes for this block, each loop will change length depending on how many stimulations fall in a sweep
    for w in np.arange(25, dtype='int') :
        hist_inds[w] = np.zeros(PSTH_spike_counts[w][0]), np.zeros(PSTH_spike_counts[w][1])
        spikecountsup = 0
        spikecountsdown = 0
        for i in np.arange(len(stimulation), dtype='int') : 
            for x in np.arange(len(stimtimes[w][0][i]), dtype='int') :     # dynamic loop depends on how many stims fall in sweep      
                timesUP = starts[i] + stimtimes[w][0][i][x]/stim_samp
                spikecountup = len(spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)])
                spikeidxup = spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)]
                #spikeidxup = ((spikeidxup - starts[i])/float(stops[i] - starts[i])*len(stimulation[i,0]))
                hist_inds[w][0][spikecountsup:(spikecountsup+spikecountup)] = spikeidxup-timesUP#stimtimes[w][0][i][x]/float(stops[i] - starts[i])*len(stimulation[i,0])
                spikecountsup += spikecountup
            
            for y in np.arange(len(stimtimes[w][1][i]), dtype='int') :     # dynamic loop depends on how many stims fall in sweep
                timesDOWN = starts[i] + stimtimes[w][1][i][y]/stim_samp                
                spikecountdown = len(spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)])
                spikeidxdown = spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)]
                #spikeidxdown = ((spikeidxdown - starts[i])/float(stops[i] - starts[i])*len(stimulation[i,0]))
                hist_inds[w][1][spikecountsdown:(spikecountsdown+spikecountdown)] = spikeidxdown-timesDOWN#stimtimes[w][1][i][y]/float(stops[i] - starts[i])*len(stimulation[i,0])
                spikecountsdown += spikecountdown
                
    return PSTH_spike_counts, hist_inds

# March 25 we took out the rounding fronm this function, to keep the actual times with respect to stim.
# April modified a little to be more precise with absolute times.

# Experiment Files and Folders

In [5]:
# In this cell you put all the information to make the code portable from computer to computer
# You have to place all the file names and experiments, then you loop whichever you want to analyse
#--------------------------------------------------------------------------------
#Experiment numbers
ExpeNum = [20,22,23,24,25,26,27,28,29,30,31,32]

#--------------------------------------------------------------------------------
#Folders for measurements and experiments (this is how we separate shanks in folders for individual analyses)
m164 = ['m1s1','m1s2','m1s3','m1s4','m1s5','m1s6','m1s7','m1s8']
m264 = ['m2s1','m2s2','m2s3','m2s4','m2s5','m2s6','m2s7','m2s8']
m364 = ['m3s1','m3s2','m3s3','m3s4','m3s5','m3s6','m3s7','m3s8']
m464 = ['m4s1','m4s2','m4s3','m4s4','m4s5','m4s6','m4s7','m4s8']

#--------------------------------------------------------------------------------
#Kwik files    

files20 = [ 'MEAS-150707-1_ele01_ele08.kwik',
            'MEAS-150707-1_ele09_ele16.kwik',
            'MEAS-150707-1_ele17_ele24.kwik',
            'MEAS-150707-1_ele25_ele32.kwik',
            'MEAS-150707-23_ele01_ele08.kwik',
            'MEAS-150707-23_ele16_ele09.kwik']#,
            #'MEAS-150707-23_ele17_ele24.kwik',  not in S2
            #'MEAS-150707-23_ele25_ele32.kwik',] not in S2

files22 = [ 'MEAS-150716-12_ele01_ele08.kwik',
            'MEAS-150716-12_ele09_ele16.kwik',
            'MEAS-150716-12_ele17_ele24.kwik',
            'MEAS-150716-12_ele25_ele32.kwik',
            'MEAS-150716-3_ele01_ele08.kwik',
            'MEAS-150716-3_ele09_ele16.kwik',
            'MEAS-150716-3_ele17_ele24.kwik',
            'MEAS-150716-3_ele25_ele32.kwik',]

files23 = [ 'MEAS-151027-1_ele01_ele08.kwik',
            'MEAS-151027-1_ele09_ele16.kwik',
            #'MEAS-151027-1_ele17_ele24.kwik', out of S2
            #'MEAS-151027-1_ele25_ele32.kwik', out of S2
            #'MEAS-151027-1_ele33_ele40.kwik', out of S2
            #'MEAS-151027-1_ele41_ele48.kwik', out of S2
            #'MEAS-151027-1_ele49_ele56.kwik', out of S2
            #'MEAS-151027-1_ele57_ele64.kwik', out of S2
            'MEAS-151027-2_ele01_ele08.kwik',
            'MEAS-151027-2_ele09_ele16.kwik',
            'MEAS-151027-2_ele17_ele24.kwik',
            'MEAS-151027-2_ele25_ele32.kwik',
            'MEAS-151027-2_ele33_ele40.kwik']#,
            #'MEAS-151027-2_ele41_ele48.kwik',  out of S2
            #'MEAS-151027-2_ele49_ele56.kwik',  out of S2
            #'MEAS-151027-2_ele57_ele64.kwik']  out of S2

files24 = [#'MEAS-151103-1_EXTRACTED_ele25_ele32.kwik',  no functional responses
           'MEAS-151103-1_EXTRACTED_ele33_ele40.kwik',  
           'MEAS-151103-1_EXTRACTED_ele41_ele48.kwik',  
           'MEAS-151103-1_EXTRACTED_ele49_ele56.kwik',  
           'MEAS-151103-1_EXTRACTED_ele57_ele64.kwik',
           #'MEAS-151103-2_ele33_ele40.kwik',   no units
           'MEAS-151103-2_ele41_ele48.kwik',
           'MEAS-151103-2_ele49_ele56.kwik',
           'MEAS-151103-2_ele57_ele64.kwik']

#OUT OF S2
files25 = [ 'MEAS-151105-1good_ele01_ele08.kwik',
            'MEAS-151105-1good_ele09_ele16.kwik',
            'MEAS-151105-1good_ele17_ele24.kwik',
            'MEAS-151105-1good_ele25_ele32.kwik',
            'MEAS-151105-1good_ele33_ele40.kwik',
            'MEAS-151105-1good_ele41_ele48.kwik',
            'MEAS-151105-1good_ele49_ele56.kwik',
            'MEAS-151105-1good_ele57_ele64.kwik',
            'MEAS-151105-2_ele01_ele08.kwik',
            'MEAS-151105-2_ele09_ele16.kwik',
            'MEAS-151105-2_ele17_ele24.kwik',
            'MEAS-151105-2_ele25_ele32.kwik',
            'MEAS-151105-2_ele33_ele40.kwik',
            'MEAS-151105-2_ele41_ele48.kwik',
            'MEAS-151105-2_ele49_ele56.kwik',
            'MEAS-151105-2_ele57_ele64.kwik']

files26 = [ 'MEAS-151110-1_ele01_ele08.kwik',
            'MEAS-151110-1_ele09_ele16.kwik',
            'MEAS-151110-1_ele17_ele24.kwik',
            'MEAS-151110-1_ele25_ele32.kwik',
            'MEAS-151110-1_ele33_ele40.kwik',
            'MEAS-151110-1_ele41_ele48.kwik',
            'MEAS-151110-1_ele49_ele56.kwik',
            #'MEAS-151110-1_ele57_ele64.kwik', out of S2
            'MEAS-151110-2_ele01_ele08.kwik',
            'MEAS-151110-2_ele09_ele16.kwik',
            'MEAS-151110-2_ele17_ele24.kwik',
            'MEAS-151110-2_ele25_ele32.kwik']#,
            #'MEAS-151110-2_ele33_ele40.kwik',  no units
            #'MEAS-151110-2_ele41_ele48.kwik', out of S2
            #'MEAS-151110-2_ele49_ele56.kwik', no units
            #'MEAS-151110-2_ele57_ele64.kwik', no units
            #'MEAS-151110-3_ele01_ele08.kwik',  out of S2
            #'MEAS-151110-3_ele09_ele16.kwik',  out of S2
            #'MEAS-151110-3_ele17_ele24.kwik',  out of S2
            #'MEAS-151110-3_ele25_ele32.kwik',  out of S2
            #'MEAS-151110-3_ele33_ele40.kwik',  out of S2
            #'MEAS-151110-3_ele41_ele48.kwik',  out of S2
            #'MEAS-151110-3_ele49_ele56.kwik',  no units
            #'MEAS-151110-3_ele57_ele64.kwik']  no units

files27  = ['MEAS-151112-1_ele01_ele08.kwik',
            'MEAS-151112-1_ele09_ele16.kwik',
            'MEAS-151112-1_ele17_ele24.kwik',
            'MEAS-151112-1_ele25_ele32.kwik',
            'MEAS-151112-1_ele33_ele40.kwik',
            'MEAS-151112-1_ele41_ele48.kwik',
            'MEAS-151112-1_ele49_ele56.kwik',
            'MEAS-151112-1_ele57_ele64.kwik',
            'MEAS-151112-2_ele01_ele08.kwik',
            'MEAS-151112-2_ele09_ele16.kwik',
            'MEAS-151112-2_ele17_ele24.kwik',
            'MEAS-151112-2_ele25_ele32.kwik',
            'MEAS-151112-2_ele33_ele40.kwik',
            'MEAS-151112-2_ele41_ele48.kwik',
            'MEAS-151112-2_ele49_ele56.kwik',
            'MEAS-151112-2_ele57_ele64.kwik',
            'MEAS-151112-3_ele01_ele08.kwik',
            'MEAS-151112-3_ele09_ele16.kwik',
            'MEAS-151112-3_ele17_ele24.kwik',
            'MEAS-151112-3_ele25_ele32.kwik',
            'MEAS-151112-3_ele33_ele40.kwik',
            'MEAS-151112-3_ele41_ele48.kwik',
            'MEAS-151112-3_ele49_ele56.kwik',
            'MEAS-151112-3_ele57_ele64.kwik']

files28 =  ['MEAS-151116-1_ele01_ele08.kwik',
            'MEAS-151116-1_ele09_ele16.kwik',
            'MEAS-151116-1_ele17_ele24.kwik',
            'MEAS-151116-1_ele25_ele32.kwik',
            'MEAS-151116-1_ele33_ele40.kwik',
            'MEAS-151116-1_ele41_ele48.kwik',
            'MEAS-151116-1_ele49_ele56.kwik',
            #'MEAS-151116-1_ele57_ele64.kwik',
            'MEAS-151116-2_ele01_ele08.kwik',
            'MEAS-151116-2_ele09_ele16.kwik',
            'MEAS-151116-2_ele17_ele24.kwik',
            'MEAS-151116-2_ele25_ele32.kwik',
            'MEAS-151116-2_ele33_ele40.kwik',
            'MEAS-151116-2_ele41_ele48.kwik',
            'MEAS-151116-2_ele49_ele56.kwik',
            'MEAS-151116-2_ele57_ele64.kwik',
            'MEAS-151116-3_ele01_ele08.kwik',
            'MEAS-151116-3_ele09_ele16.kwik',
            'MEAS-151116-3_ele17_ele24.kwik',
            'MEAS-151116-3_ele25_ele32.kwik',
            'MEAS-151116-3_ele33_ele40.kwik',
            'MEAS-151116-3_ele41_ele48.kwik',
            'MEAS-151116-3_ele49_ele56.kwik',
            'MEAS-151116-3_ele57_ele64.kwik']

files29  = ['MEAS-151118-1_ele01_ele08.kwik',
            'MEAS-151118-1_ele09_ele16.kwik',
            'MEAS-151118-1_ele17_ele24.kwik',
            'MEAS-151118-1_ele25_ele32.kwik',
            'MEAS-151118-1_ele33_ele40.kwik',
            'MEAS-151118-1_ele41_ele48.kwik',
            'MEAS-151118-1_ele49_ele56.kwik',
            'MEAS-151118-1_ele57_ele64.kwik',
            'MEAS-151118-2_ele01_ele08.kwik',
            'MEAS-151118-2_ele09_ele16.kwik',
            'MEAS-151118-2_ele17_ele24.kwik',
            'MEAS-151118-2_ele25_ele32.kwik',
            'MEAS-151118-2_ele33_ele40.kwik',
            'MEAS-151118-2_ele41_ele48.kwik',
            'MEAS-151118-2_ele49_ele56.kwik',
            'MEAS-151118-2_ele57_ele64.kwik',
            'MEAS-151118-3_ele01_ele08.kwik',
            'MEAS-151118-3_ele09_ele16.kwik',
            'MEAS-151118-3_ele17_ele24.kwik',
            'MEAS-151118-3_ele25_ele32.kwik',
            'MEAS-151118-3_ele33_ele40.kwik',
            'MEAS-151118-3_ele41_ele48.kwik',
            'MEAS-151118-3_ele49_ele56.kwik',
            'MEAS-151118-3_ele57_ele64.kwik']


files30  = ['MEAS-151208-2_ele01_ele08.kwik',
            'MEAS-151208-2_ele09_ele16.kwik',
            'MEAS-151208-2_ele17_ele24.kwik',
            'MEAS-151208-2_ele25_ele32.kwik',
            'MEAS-151208-2_ele33_ele40.kwik',
            'MEAS-151208-2_ele41_ele48.kwik',
            'MEAS-151208-2_ele49_ele56.kwik',
            'MEAS-151208-2_ele57_ele64.kwik',
            'MEAS-151208-3_ele01_ele08.kwik',
            'MEAS-151208-3_ele09_ele16.kwik',
            'MEAS-151208-3_ele17_ele24.kwik',
            'MEAS-151208-3_ele25_ele32.kwik',
            'MEAS-151208-3_ele33_ele40.kwik',
            'MEAS-151208-3_ele41_ele48.kwik',
            'MEAS-151208-3_ele49_ele56.kwik',
            'MEAS-151208-3_ele57_ele64.kwik',
            'MEAS-151208-4_ele01_ele08.kwik',
            'MEAS-151208-4_ele09_ele16.kwik',
            'MEAS-151208-4_ele17_ele24.kwik',
            'MEAS-151208-4_ele25_ele32.kwik',
            'MEAS-151208-4_ele33_ele40.kwik',
            'MEAS-151208-4_ele41_ele48.kwik',
            'MEAS-151208-4_ele49_ele56.kwik',
            'MEAS-151208-4_ele57_ele64.kwik',
            'MEAS-151208-5_ele01_ele08.kwik',
            'MEAS-151208-5_ele09_ele16.kwik',
            'MEAS-151208-5_ele17_ele24.kwik',
            'MEAS-151208-5_ele25_ele32.kwik',
            'MEAS-151208-5_ele33_ele40.kwik',
            'MEAS-151208-5_ele41_ele48.kwik',
            'MEAS-151208-5_ele49_ele56.kwik',
            'MEAS-151208-5_ele57_ele64.kwik']


files31 = [ #'MEAS-151210-1_ele01_ele08.kwik',  no units
            #'MEAS-151210-1_ele09_ele16.kwik',  no units
            #'MEAS-151210-1_ele17_ele24.kwik',  no units
            #'MEAS-151210-1_ele25_ele32.kwik',  no units
            #'MEAS-151210-1_ele33_ele40.kwik',  no units
            'MEAS-151210-1_ele41_ele48.kwik',
            'MEAS-151210-1_ele49_ele56.kwik',
            'MEAS-151210-1_ele57_ele64.kwik',
            #'MEAS-151210-2_ele01_ele08.kwik',   out of S2
            #'MEAS-151210-2_ele09_ele16.kwik',   out of S2
            #'MEAS-151210-2_ele17_ele24.kwik',   out of S2
            #'MEAS-151210-2_ele25_ele32.kwik',   out of S2
            'MEAS-151210-2_ele33_ele40.kwik',
            'MEAS-151210-2_ele41_ele48.kwik',
            'MEAS-151210-2_ele49_ele56.kwik',
            'MEAS-151210-2_ele57_ele64.kwik',
            #'MEAS-151210-3_ele01_ele08.kwik',   out of S2
            #'MEAS-151210-3_ele09_ele16.kwik',   no units
            #'MEAS-151210-3_ele17_ele24.kwik',   no units
            #'MEAS-151210-3_ele25_ele32.kwik',   out of S2
            'MEAS-151210-3_ele33_ele40.kwik',
            'MEAS-151210-3_ele41_ele48.kwik',
            'MEAS-151210-3_ele49_ele56.kwik',
            'MEAS-151210-3_ele57_ele64.kwik']

files32 = [ 'MEAS-151214-1_ele01_ele08.kwik',
            'MEAS-151214-1_ele09_ele16.kwik',
            'MEAS-151214-1_ele17_ele24.kwik',
            'MEAS-151214-1_ele25_ele32.kwik',
            'MEAS-151214-1_ele33_ele40.kwik',
            'MEAS-151214-2_ele01_ele08.kwik',
            'MEAS-151214-2_ele09_ele16.kwik',
            'MEAS-151214-2_ele17_ele24.kwik',
            'MEAS-151214-2_ele25_ele32.kwik',
            'MEAS-151214-2_ele33_ele40.kwik']

#--------------------------------------------------------------------------------
#--------------------------------------------------------------------------------
# Here I create my dictionary of experiments
Expe={}
Vtags={}
Stim={}
for num in ExpeNum: 
    Expe[num] = dict()
    Vtags[num] = dict()
#---------------------------------------
i=0        
for meas in np.append(m164[0:4],m364[0:2]):
    Expe[20][meas] = files20[i]
    i+=1
#---------------------------------------
i=0 
for meas in np.append(m164[0:4],m364[0:4]):
    Expe[22][meas] = files22[i]
    i+=1
#---------------------------------------
i=0 
for meas in np.append(m164[0:2],m264[0:5]):
    Expe[23][meas] = files23[i]
    i+=1
#---------------------------------------
i=0
for meas in np.append(m164[4:8],m264[5:8]):    
    Expe[24][meas] = files24[i]
    i+=1
#---------------------------------------
i=0 
for meas in np.append(m164,m264):    
    Expe[25][meas] = files25[i]
    i+=1
#---------------------------------------
i=0
for meas in np.append(m164[0:7],m264[0:4]):
    Expe[26][meas] = files26[i]
    i+=1
#---------------------------------------
i=0
for meas in np.append(np.append(m164[0:7],m264),m364):    
    Expe[28][meas] = files28[i]
    i+=1
#---------------------------------------    
i=0
for meas in np.append(np.append(m164,m264),m364):    
    Expe[27][meas] = files27[i]
    Expe[29][meas] = files29[i]
    Expe[30][meas] = files30[i]
    i+=1
#---------------------------------------    
for meas in m464:
    Expe[30][meas] = files30[i]
    i+=1
i=0
#---------------------------------------    
i=0
for meas in np.append(np.append(m164[5:8],m264[4:8]),m364[4:8]):    
    Expe[31][meas] = files31[i]
    i+=1
i=0
#---------------------------------------    
for meas in np.append(m164[0:5],m264[0:5]):
    Expe[32][meas] = files32[i]
    i+=1

#--------------------------------------------------------------------------------
#Vtag files 
Vtags[20] = ['MEAS-150707-1_Vtag1.dat','nada','MEAS-150707-23_Vtag1.dat']
Vtags[22] = ['MEAS-150716-12_Vtag1.dat','nada','MEAS-150716-3_Vtag1.dat']
Vtags[23] = ['MEAS-151027-1_Vtag1.dat','MEAS-151027-2_Vtag1.dat']
Vtags[23] = ['MEAS-151027-1_Vtag1.dat','MEAS-151027-2_Vtag1.dat']
Vtags[24] = ['MEAS-151103-1_Vtag1.dat','MEAS-151103-2_Vtag1.dat']
Vtags[25] = ['MEAS-151105-1good_Vtag1.dat','MEAS-151105-2_Vtag1.dat']
Vtags[26] = ['MEAS-151110-1_Vtag1.dat','MEAS-151110-2_Vtag1.dat','MEAS-151110-3_Vtag1.dat']
Vtags[27] = ['MEAS-151112-1_Vtag1.dat','MEAS-151112-2_Vtag1.dat','MEAS-151112-3_Vtag1.dat']
Vtags[28] = ['MEAS-151116-1_Vtag1.dat','MEAS-151116-2_Vtag1.dat','MEAS-151116-3_Vtag1.dat']
Vtags[29] = ['MEAS-151118-1_Vtag1.dat','MEAS-151118-2_Vtag1.dat','MEAS-151118-3_Vtag1.dat']
Vtags[30] = ['MEAS-151208-2_Vtag1.dat','MEAS-151208-3_Vtag1.dat','MEAS-151208-4_Vtag1.dat','MEAS-151208-5_Vtag1.dat']
Vtags[31] = ['MEAS-151210-1_Vtag1.dat','MEAS-151210-2_Vtag1.dat','MEAS-151210-3_Vtag1.dat']
Vtags[32] = ['MEAS-151214-1_Vtag1.dat','MEAS-151214-2_Vtag1.dat']

#--------------------------------------------------------------------------------
#Stimulus type
for i in range(23,33):
    Stim[i] = 'big_STIM_FC_corrected'
for i in range(15,23):
    Stim[i] = 'big_STIM'
for i in range(10,15):
    Stim[i] = 'small_STIM'  
    
#--------------------------------------------------------------------------------
#Root folder to work in, such all will be in subfolders 
#e.g.: "/EXP_23/m1s1/" for data or "/STIM/" for stims
rootF = '/home/matias/WORKSPACE/'    
stimFolder = rootF +'STIM/'

#I have a separate folder for exp 22 and before
rootF_kwiks = rootF    #uncomment this to work with the other root folder
#rootF_kwiks = '/media/matias/DATA/WORKSPACE2/'

#--------------------------------------------------------------------------------
#Initialize wave features dictionary
PSTHdata = dict()

# Define and load data files from experiments

In [7]:
global binname, textname
#---------------------------------------------------------------------------------------
SelExp = [22]#[23,24,26,27,28,29,30,31,32]#[20,22,23,24,26,27,28,29,30,31,32] #Expe                                        #select experiment numbers!
grupete = [2]   #select cluster groups! 2 for good clusters 1 for multiunits, 3 for unsorted

#select measurement and/or shanks!
#Measurements = m264[7:8]           #['m1s1']#['m3s1','m3s3']#m12[-4:]#['m1s1','m1s2','m1s3','m1s4']   

#select type of stimuli, for PSTH is only 'F' and this does not change anything
choices = ['F','C','U']                      #select stimulus type (for STA and STC)

# choice code not to ploteverything at the same time
ploteo = [1,0,0,0]                                           #1 to make plots: psth,sta,ufc,stc

dirs =[]
#--------------------------------------------------------------------------------
# Loop Experiments
#--------------------------------------------------------------------------------
last_exp=0     #we use this to load stim only when we change experiment
for expe in SelExp:
    
    PSTHdata = {}
    PSTH_spikes_counts = {}
    
    Measurements = sorted(Expe[expe])                         #uncommento to select all
    print(expe)

    #This two lines are to account for diffrerent stims when looping diffrerent experiments
    binname= stimFolder + Stim[expe] + '/Stimulus_UCC.bin'     
    textname=stimFolder + Stim[expe] + '/Stimulus_UCC.txt'
    
    last_meas =0   #we use this to find when we change measurement to load Vtag and stim again
    
    #--------------------------------------------------------------------------------
    #loop goodunits, multiunits, unsorted...
    for group in grupete:   #2 for good clusters 1 for multiunits 3 for unsorted
        #folder names
        if group ==3:
            dirs  = [rootF + 'OUTPUT/PDFallRM/EXP_'+str(expe),rootF + 'OUTPUT/PDFallRM/STC/EXP_'+str(expe)]
        if group ==2:
            dirs  = [rootF + 'OUTPUT/PDFall/EXP_'+str(expe),rootF + 'OUTPUT/PDFall/STC/EXP_'+str(expe)]
        if group ==1:
            dirs  = [rootF + 'OUTPUT/PDFallM/EXP_'+str(expe),rootF + 'OUTPUT/PDFallM/STC/EXP_'+str(expe)]
        print('group:', group)
        #--------------------------------------------------------------------------------
        #loop measurements and shanks
        #Measurements = sorted((Expe[expe]))
        
        for meas in Measurements:           
            print(meas)
            current_meas = int(meas[1])   #measurement number
            #---------------------------------------------------------------
            #select datafile
            sp_file = rootF_kwiks + 'EXP_' + str(expe) +'/Spike_Sorting/'+ meas +'/'+ Expe[expe][meas]
            #load datafile
            Spikes, sampling_freq = readkwikinfo(sp_file, group)  
            #---------------------------------------------------------------
            #load stimulus if looping new experiment, without trimming
            if expe!=last_exp or (expe==20 and meas=='m3s1') or (expe==22 and meas=='m3s1'):
                stimraw = []
                stimtyperaw=[]
                print("   reading stim at:",  meas)
                stimraw,stimtyperaw = read_stimulus(expe,meas)

            #---------------------------------------------------------------
            #load Vtag if looping new measurement
            if (last_meas!=current_meas): #or (expe!=last_exp):   
                # get Vtag name
                measV=int(meas[1])-1
                bin_file = rootF_kwiks + 'EXP_' + str(expe) +'/' + Vtags[expe][measV]
                #-----------------------------------------------------------
                Vtag1 =[]                
                Vtag1 = np.fromfile(file=bin_file, dtype=np.int16)
                # here we trim down stim and stimtype from Vtag1 information
                print("   trimming stim at:", meas)
                stim = []
                stimtype=[]
                stim, stimtype,starts, stops = readVtag(Vtag1,stimraw,stimtyperaw)
                                          
            last_meas = current_meas     #update measurement variable
            last_exp = expe              #update experiment variable
            
            if len(Spikes.keys())>0:                              #do only if there are clusters
                #Build PSTHs
                t_before = .100  
                t_after = .100
                
                print('   building psths')
                #PSTH_spikes_counts, hist_output = BuildPSTH(stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops,expe,meas)
                PSTH_spikes_counts= BuildPSTH(stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops,expe,meas)
           
            PSTHdata.update(PSTH_spikes_counts)
            #print(sorted(list(PSTHdata.keys())))

            #PSTHdata.update(hist_output)
    
    print('   saving')
    if group == 2:
        filesave =rootF +'S2_git/data/'+'psthdata' + str(expe)
    else:
        filesave =rootF +'S2_git/data/'+ 'psthdataMultiR' + str(expe)
        
    save_obj(PSTHdata,filesave)

22
group: 2
m1s1
   reading stim at: m1s1
   trimming stim at: m1s1
m1s2
   building psths
m1s3
   building psths
m1s4
   building psths
m3s1
   reading stim at: m3s1
   trimming stim at: m3s1
   building psths
m3s2
   building psths
m3s3
   building psths
m3s4
   building psths
   saving


In [34]:
#filesave

sorted(list(PSTHdata.keys()))
PSTHdata['exp20_m1s1_c29'].keys()

data = rootF +'S2_git/data/'+'psthdata' + str(22)

a = load_obj(data)

a['exp22_m3s4_c1138'].keys()

dict_keys(['psth_counts', 'meas', 'psth_length', 'clusnum', 'psth_times', 'exp', 'shank'])

In [ ]:
#update spiketimes to wavedataAll
SelExp = [20] #[22,24,26,27,28,29,30,31,32] #Expe                                        #select experiment numbers!

Folder = '/home/matias/WORKSPACE/S2_git/data'    

for e in SelExp:

    data = Folder +'/wavedata'+ str(e)
    wavedata = load_obj(data)

    idx = list(wavedata.keys())

    datapsth = Folder +'/psthdata' + str(e)
    psthdata = load_obj(datapsth)


    for n in idx:
        wavedata[n].update(psthdata[n])    
            
    data = Folder +'data/datamix' + str(e)

    save_obj(wavedata, data)



In [ ]:
idx = list(PSTHdata.keys())
PSTHdata[idx[0]].keys()
#PSTHdata[idx[0]]


In [ ]:
Spikes.keys()

In [ ]:
PSTHdata[idx[0]][1]